<a href="https://colab.research.google.com/github/Elhai12/SBA_loans_predict/blob/main/Preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [73]:
import pandas as pd
import numpy as np
import seaborn as sns
import warnings
import re
warnings.filterwarnings('ignore')


In [74]:
df_loans = pd.read_csv('/content/SBAnational.csv')
df_filter = df_loans[df_loans['State'].isin(['FL'])]
df_filter.info()

<class 'pandas.core.frame.DataFrame'>
Index: 41212 entries, 4 to 898928
Data columns (total 27 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   LoanNr_ChkDgt      41212 non-null  int64  
 1   Name               41212 non-null  object 
 2   City               41212 non-null  object 
 3   State              41212 non-null  object 
 4   Zip                41212 non-null  int64  
 5   Bank               41159 non-null  object 
 6   BankState          41159 non-null  object 
 7   NAICS              41212 non-null  int64  
 8   ApprovalDate       41212 non-null  object 
 9   ApprovalFY         41212 non-null  object 
 10  Term               41212 non-null  int64  
 11  NoEmp              41212 non-null  int64  
 12  NewExist           41207 non-null  float64
 13  CreateJob          41212 non-null  int64  
 14  RetainedJob        41212 non-null  int64  
 15  FranchiseCode      41212 non-null  int64  
 16  UrbanRural         41212 n

In [75]:
df_filter.to_csv("sba_loans_filtered",index=False)

In [76]:
df_filter['Sector'] = df_filter['NAICS'].astype(str).str[:2]
# df_filter['Sector'].value_counts()

In [77]:
df_filter['Region'] = pd.cut(
    df_filter['Zip'].astype(int),
    bins=[32000, 32399, 32799, 33199, 35000],
    labels=['North-Central Florida', 'Central Florida', 'Central-East Florida', 'South Florida']
)

In [78]:
df_filter['Region'].value_counts()

,count
Region,
South Florida,17823
Central-East Florida,14290
North-Central Florida,4654
Central Florida,4377


In [79]:
df_filter.columns

Index(['LoanNr_ChkDgt', 'Name', 'City', 'State', 'Zip', 'Bank', 'BankState',
       'NAICS', 'ApprovalDate', 'ApprovalFY', 'Term', 'NoEmp', 'NewExist',
       'CreateJob', 'RetainedJob', 'FranchiseCode', 'UrbanRural', 'RevLineCr',
       'LowDoc', 'ChgOffDate', 'DisbursementDate', 'DisbursementGross',
       'BalanceGross', 'MIS_Status', 'ChgOffPrinGr', 'GrAppv', 'SBA_Appv',
       'Sector', 'Region'],
      dtype='object')

In [80]:
df_filter['Is_Franchise'] = df_filter['FranchiseCode'].apply(lambda x: 0 if (x == '00001') or (x == '00000') else 1)

In [81]:
df_drop = df_filter.drop(['Name','City','State','BankState','NAICS','Zip','LoanNr_ChkDgt','ChgOffDate','BalanceGross','ChgOffPrinGr','FranchiseCode'],axis=1)

In [82]:
df_drop.columns

Index(['Bank', 'ApprovalDate', 'ApprovalFY', 'Term', 'NoEmp', 'NewExist',
       'CreateJob', 'RetainedJob', 'UrbanRural', 'RevLineCr', 'LowDoc',
       'DisbursementDate', 'DisbursementGross', 'MIS_Status', 'GrAppv',
       'SBA_Appv', 'Sector', 'Region', 'Is_Franchise'],
      dtype='object')

In [83]:
print(df_drop['Bank'].unique().tolist())

['FLORIDA BUS. DEVEL CORP', 'REGIONS BANK', 'CENTENNIAL BANK', 'BANESCO USA', 'CITIBANK, N.A.', 'WELLS FARGO BANK NATL ASSOC', 'THE BANK OF TAMPA', 'PNC BANK, NATIONAL ASSOCIATION', 'SYNOVUS BANK', 'BANCO POPULAR NORTH AMERICA', 'READYCAP LENDING, LLC', 'CALIFORNIA BANK & TRUST', 'BANK OF AMERICA NATL ASSOC', 'SUNTRUST BANK', 'LOANS FROM OLD CLOSED LENDERS', 'JPMORGAN CHASE BANK NATL ASSOC', 'FLORIDA 1ST CAP. FINAN CORPORA', 'STEARNS BK NATL ASSOC', 'BBCN BANK', 'SOUTHERN COMM. BANK NATL ASSOC', 'BANKFIRST', 'NEWTEK SMALL BUS. FINANCE INC.', 'C1 BANK', 'BUSINESS DEVEL CORP OF N.E. FL', 'BUSINESS LOAN CENTER, LLC', 'GE CAP. SMALL BUS. FINAN CORP', 'TD BANK, NATIONAL ASSOCIATION', '1ST UNITED BANK', 'CAPITAL ONE NATL ASSOC', 'BRANCH BK. & TR CO', 'OLD FLORIDA BANK', 'AMER BUS. LENDING INC.', 'CENTERSTATE BK OF FLORIDA NATL', 'HARBOR COMMUNITY BANK', 'INDEPENDENT DEVEL SERVICES COR', 'EVERBANK', 'GULFCOAST BUS. FINANCE INC.', 'FIFTH THIRD BANK', 'CHARLOTTE STATE BANK & TRUST', 'TRUSTMARK 

In [84]:
def categorize_bank(name):
    name = str(name).upper()
    if re.search(r'\bBANK\b', name):
        return "Commercial Bank"

    elif re.search(r'\b(CU|CREDIT UNION|CREDIT|SAVINGS|LOAN|MEMBERSHIP)\b', name):
        return "Credit Union"

    elif re.search(r'\b(INTERNATIONAL|GLOBAL|OVERSEAS|WORLD)\b', name):
        return "International"


    elif re.search(r'\b(CAPITAL|FINANCE|INVESTMENT|SECURITIES|CORPORATION|GROUP)\b', name):
        return "Capital/Finance Institution"


    elif re.search(r'\b(DEVELOPMENT|BUSINESS|SBA|SMALL BUSINESS|ENTERPRISE)\b', name):
        return "Development Institution"


    else:
        return "Other"


df_drop['Category_bank'] = df_drop['Bank'].apply(categorize_bank)


In [85]:
df_drop['Category_bank'].value_counts()

,count
Category_bank,
Commercial Bank,27507
Other,9903
Capital/Finance Institution,2437
Credit Union,877
Development Institution,470
International,18


In [86]:
df_drop_bank = df_drop.drop(['Bank'],axis=1)

In [87]:
df_drop_bank.to_csv("sba_loans_flat",index=False)